In [ ]:
%load_ext autoreload
%autoreload 2

PREDICTION_PATH = '../data/predictions/'

#### Load data

In [ ]:
from datafile_methods.data_io import load_datasets, load_datasets_sur

# Load datasets
train, test, validation, ratings, sample_submission = load_datasets()
train_sur, test_sur, validation_sur, ratings_sur, sample_submission_sur = load_datasets_sur()

#### Train models

Baselines

In [ ]:
from prediction_methods.baseline_model import model_baseline

# Train on `train`. Predict for `test` and `validation` datasets
model_baseline(train, test, True, validation_data=validation,
    prediction_path=PREDICTION_PATH)

# Train on entire training set. Predict for `submission` dataset
model_baseline(ratings, sample_submission, False, prediction_path=PREDICTION_PATH)

# Test RMSE of model_baseline: 1.0095626730173732
#### BEFORE REFACTORING #######
#test_rmse, pred_test = model_baseline(train, test, True, verbose=True)
## Test RMSE of baseline using baseline: 1.0057078177840961
#model_baseline(ratings_csr, sample_submission_csr, False, True, "tmp")
## Achieves 1.00386 in Kaggle

Matrix factorization with ALS

In [ ]:
from prediction_methods.mf_als_model import model_mf_als

# Train on `train`. Predict for `test` and `validation` datasets
model_mf_als(train, test, True, validation_data=validation,
    prediction_path=PREDICTION_PATH, k=20, lambda_u=.1, lambda_i=.1,
    tol=1e-6, max_iter=100, fn_suffix='_k20')

# Train on entire training set. Predict for `submission` dataset
model_mf_als(ratings, sample_submission, False, prediction_path=PREDICTION_PATH,
    k=20, lambda_u=.1, lambda_i=.1, tol=1e-6, max_iter=100, fn_suffix='_k20')

## Before refactoring achieved 0.9878 in training
## Other good param_combs are (k20, lambdas 0.095) and (k30, lambdas 0.095)

Matrix factorization with SVD (scipy)

In [ ]:
from prediction_methods.mf_svd_model import model_mf_svd

# Train on `train`. Predict for `test` and `validation` datasets
model_mf_svd(train, test, True, validation_data=validation,
    prediction_path=PREDICTION_PATH, k=16, library='scipy', fn_suffix='_scipy')

# Train on entire training set. Predict for `submission` dataset
model_mf_svd(ratings, sample_submission, False, prediction_path=PREDICTION_PATH,
    k=16, library='scipy', fn_suffix='_scipy')

## Before ref. achieved 0.99385 on test. It's better to average it with the next ones!

Matrix factorization with SVD (sklearn)

In [ ]:
# Train on `train`. Predict for `test` and `validation` datasets
model_mf_svd(train, test, True, validation_data=validation, prediction_path=PREDICTION_PATH,
    k=12, n_iter=50, library='sklearn', random_state=70, fn_suffix='_sklearn')

# Train on entire training set. Predict for `submission` dataset
model_mf_svd(ratings, sample_submission, False, prediction_path=PREDICTION_PATH,
    k=12, n_iter=50, library='sklearn', random_state=70, fn_suffix='_sklearn')

### Bef ref. achieved 0.99386 (approx) on test
# Achieves 0.99082 in Kaggle

Recommend's ALS model

In [ ]:
from prediction_methods.recommend_model import model_mf_als_recommend
n_item, n_user = ratings.shape

# Train on `train`. Predict for `test` and `validation` datasets
model_mf_als_recommend(train, test, True, n_user, n_item,
    prediction_path=PREDICTION_PATH, validation_data=validation, k=20,
    n_iter=50, reg=85e-3)

# Train on entire training set. Predict for `submission` dataset
model_mf_als_recommend(ratings, sample_submission, False,
    n_user, n_item, prediction_path=PREDICTION_PATH,
    k=20, n_iter=50, reg=85e-3)

#Test RMSE of model_mf_als_recommend: 0.9975413590062877
#0.98585 on Kaggle

Surprise models

* SlopeOne

In [ ]:
from prediction_methods.surprise_models import model_slope_one

In [ ]:
model_slope_one(train_sur, test_sur, True,
    prediction_path=PREDICTION_PATH, validation_data=validation_sur)

In [ ]:
model_slope_one(ratings_sur, sample_submission_sur, False,
    prediction_path=PREDICTION_PATH)

* CoClustering

In [ ]:
from prediction_methods.surprise_models import model_co_clustering
model_co_clustering(train_sur, test_sur, True,
    prediction_path=PREDICTION_PATH, validation_data=validation_sur,
    n_cltr_u=75, n_cltr_i=3, n_epochs=100)

model_co_clustering(ratings_sur, sample_submission_sur, False,
    prediction_path=PREDICTION_PATH,
    n_cltr_u=75, n_cltr_i=3, n_epochs=100)

* KNN Baseline (user based)

In [ ]:
from prediction_methods.surprise_models import model_knn_baseline
model_knn_baseline(train_sur, test_sur, True,
    prediction_path=PREDICTION_PATH, validation_data=validation_sur,
    k=300, min_k=20, name='pearson_baseline', user_based=True,
    fn_suffix='_u')

model_knn_baseline(ratings_sur, sample_submission_sur, False,
    prediction_path=PREDICTION_PATH,
    k=300, min_k=20, name='pearson_baseline', user_based=True,
    fn_suffix='_u')

* KNN Baseline (item based)

In [ ]:
model_knn_baseline(train_sur, test_sur, True,
    prediction_path=PREDICTION_PATH, validation_data=validation_sur,
    k=120, min_k=20, name='pearson_baseline', user_based=False,
    fn_suffix='_i')

model_knn_baseline(ratings_sur, sample_submission_sur, False,
    prediction_path=PREDICTION_PATH,
    k=120, min_k=20, name='pearson_baseline', user_based=False,
    fn_suffix='_i')

#### Combine ratings

Load ratings for test and validation files

In [ ]:
#TODO

Do (regularized?) linear regression on the test rating predictions to obtain best weights

In [ ]:
#TODO

Adjust regularization for getting similar errors in test and validation

In [ ]:
#TODO
#Also ask a TA if this is ok. Also, maybe I should not be calling the sets `test` and `validation`

Load ratings for submission files

In [ ]:
#TODO

Apply previous weights to obtain a final submission

In [ ]:
#TODO